In [55]:
import pandas as pd
from influxdb import DataFrameClient

In [56]:
import requests
import pandas as pd
from datetime import datetime

In [57]:
class apiData:
    def __init__(self,index,rad,uv,press,temp,hum):
        self.index = index
        self.rad = rad
        self.uv = uv
        self.press = press
        self.temp = temp
        self.hum = hum

In [77]:
result = requests.get('https://api.weatherbit.io/v2.0/current?city=SfantuGheorghe&country=RO&key=API_KEY')

In [59]:
print(result.json())

{'data': [{'rh': 44, 'pod': 'd', 'lon': 25.78333, 'pres': 956.112, 'timezone': 'Europe/Bucharest', 'ob_time': '2020-05-19 11:44', 'country_code': 'RO', 'clouds': 100, 'ts': 1589888687, 'solar_rad': 213.715, 'state_code': '15', 'city_name': 'Sfântu-Gheorghe', 'wind_spd': 1.99177, 'last_ob_time': '2020-05-19T11:24:00', 'wind_cdir_full': 'west-northwest', 'wind_cdir': 'WNW', 'slp': 1017.17, 'vis': 24.1353, 'h_angle': 11.3, 'sunset': '17:45', 'dni': 900.7, 'dewpt': 11.7, 'snow': 0, 'uv': 2.39278, 'precip': 0, 'wind_dir': 293, 'sunrise': '02:39', 'ghi': 854.86, 'dhi': 115.19, 'aqi': 23, 'lat': 45.86667, 'weather': {'icon': 'c04d', 'code': '804', 'description': 'Overcast clouds'}, 'datetime': '2020-05-19:11', 'temp': 23.7, 'station': 'F2979', 'elev_angle': 62.31, 'app_temp': 23.3}], 'count': 1}


In [60]:
import numpy
import json

In [61]:
results = result.json()
print(results['data'][0])


{'rh': 44, 'pod': 'd', 'lon': 25.78333, 'pres': 956.112, 'timezone': 'Europe/Bucharest', 'ob_time': '2020-05-19 11:44', 'country_code': 'RO', 'clouds': 100, 'ts': 1589888687, 'solar_rad': 213.715, 'state_code': '15', 'city_name': 'Sfântu-Gheorghe', 'wind_spd': 1.99177, 'last_ob_time': '2020-05-19T11:24:00', 'wind_cdir_full': 'west-northwest', 'wind_cdir': 'WNW', 'slp': 1017.17, 'vis': 24.1353, 'h_angle': 11.3, 'sunset': '17:45', 'dni': 900.7, 'dewpt': 11.7, 'snow': 0, 'uv': 2.39278, 'precip': 0, 'wind_dir': 293, 'sunrise': '02:39', 'ghi': 854.86, 'dhi': 115.19, 'aqi': 23, 'lat': 45.86667, 'weather': {'icon': 'c04d', 'code': '804', 'description': 'Overcast clouds'}, 'datetime': '2020-05-19:11', 'temp': 23.7, 'station': 'F2979', 'elev_angle': 62.31, 'app_temp': 23.3}


In [62]:
r = result.json()['data'][0]['solar_rad']
u = result.json()['data'][0]['uv']
p = result.json()['data'][0]['pres']
t = result.json()['data'][0]['temp']
h = result.json()['data'][0]['rh']
index = datetime.now()

In [63]:
data = []

In [64]:
data = apiData(index,r, u, p, t, h)
print(data.index,data.rad, data.uv,data.press,data.temp,data.hum)

2020-05-19 11:46:09.023255 213.715 2.39278 956.112 23.7 44


In [65]:
import pandas as pd
import time
!pip install influxdb

!pip install requests
from influxdb import DataFrameClient

In [66]:
df = pd.DataFrame(columns=['index','radiation','UV','press','temp','hum'])
print (df)


Empty DataFrame
Columns: [index, radiation, UV, press, temp, hum]
Index: []


In [67]:
df.loc[0] = [data.index,data.rad,data.uv,data.press,data.temp,data.hum]

In [68]:
print(df)

                       index  radiation       UV    press  temp hum
0 2020-05-19 11:46:09.023255    213.715  2.39278  956.112  23.7  44


In [69]:
user = 'root'
password = 'root'
host='influxdb' #172.28.1.2 #localhost #3.14.5.146 - kozos osztaly InfluxDB szerver
port=8086
dbname='forecast' #az en adatbazis nevem
protocol = 'line' #json
measurement='teszt'
client = DataFrameClient(host, port, user, password, dbname)

In [70]:
client.drop_database(dbname)
client.drop_retention_policy(dbname)
client.create_database(dbname)
client.create_retention_policy(dbname, '1000d', 1, default=True)

In [71]:
client.query('show databases')

ResultSet({'('databases', None)': [{'name': '_internal'}, {'name': 'base'}, {'name': 'covid19Romania'}, {'name': 'forecast2'}, {'name': 'forecast'}]})

In [72]:
df = pd.DataFrame(df).set_index('index')

In [73]:
df.index = pd.to_datetime(df.index)
df['radiation']=df['radiation'].astype(float)
df['UV']=df['UV'].astype(float)
df['press']=df['press'].astype(float)
df['temp']=df['temp'].astype(float)
df['hum']=df['hum'].astype(float)

In [74]:
client.write_points(df, 'database', protocol=protocol,field_columns=['radiation','UV','press','temp','hum'])


True

In [1]:
for i in range(5000):
    #now = pd.to_datetime('index')
    #value=randrange(10)
    #print(now)
    #result = requests.get('https://api.weatherbit.io/v2.0/current?city=SfantuGheorghe&country=RO&key=API_KEY')
    time.sleep(2)
    index = datetime.now()
    #data={'a':{'index':now,'value':value,'szin':'piros','nev':'Denes'}}
    #df=pd.DataFrame(data).T.set_index('index')
    #df.index=pd.to_datetime(df.index)
    #df['value']=df['value'].astype(int)
    #client.write_points(df, measurement, protocol=protocol,
     #                   field_columns=['value'],
     #                   tag_columns=['szin','nev'])
    df.index = pd.to_datetime(df.index)
    df['radiation']=df['radiation'].astype(float)
    df['UV']=df['UV'].astype(float)
    df['press']=df['press'].astype(float)
    client.write_points(df, 'database', protocol=protocol,field_columns=['radiation','UV','press'])
    print(df)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-2678cb83ffa5>", line 6, in <module>
    time.sleep(2)
NameError: name 'time' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1148, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/ultratb.py", line 316, 

NameError: name 'time' is not defined